## Import necessary libraries

In [1]:
import ee
import geemap
import os
import datetime as dt
import numpy as np

In [19]:
!pip install ffmpeg-python

In [4]:
ee.Authenticate()

Enter verification code:  4/1ATx3LY6qA7wRCh4QxrC1cf3lJsmHlL6vnSQzE3Rf8i4YOoMAEloMOrce5Xg



Successfully saved authorization token.


In [2]:
ee.Initialize()

## Enter region of interest and date

In [61]:
# Mukund's ROI
hs = "Arctic"

if hs == "Arctic":
    bbox = [-10, 72.6, 12, 79.2]
    start_date = "2023-04-01"
    end_date = "2023-05-31"
    orbits = []
    # Approximately (72.6 to 79.2 N) and (-10 to 12 E) April to May 2023
elif hs == "Antarctic":
    bbox = [-176.3, -76.3, -153.9, -74.4]
    start_date = "2023-11-01"
    end_date = "2023-12-31"
    orbits = [111, 97, 112, 140, 141, 9, 38, 39, 67] #170, 120, 171, 24
    # Approximately (-76.3 to -74.4 S) and (-176.3 to -153.9 W) November - December 2023

region = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]) #ee.Geometry.Rectangle(bbox)
point = ee.Geometry.Point([np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]).buffer(20000)

## Generate GIF time series

In [62]:
# Define arguments for animation function parameters.
video_args = {
    "dimensions": 700,
    "region": region,
    "framesPerSecond": 1,
    "bands": ["HH"],
    "crs":'EPSG:3857',
    "min": -25,
    "max": -5,
    "gamma": 1    
}

collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH')).select('HH')

if len(orbits) > 0:
    collection = collection.filter(ee.Filter.inList('relativeOrbitNumber_start', orbits))
# .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
#         ee.Filter.eq('relativeOrbitNumber_start', 136)))

images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency="day", reducer="median"
)

s = images.aggregate_array('system:time_start').getInfo()
days = []
for sec in s:
    d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
    days.append(d.strftime("%Y-%m-%d"))

print(start_date, end_date, images.size().getInfo())

work_dir = "D:\\Floes\\S1"
out_gif = os.path.join(work_dir, f"S1_{hs}_{start_date}_{end_date}.gif")
geemap.download_ee_video(images, video_args, out_gif)

texted_gif = os.path.join(work_dir, f"S1_{hs}_{start_date}_{end_date}_text.gif")
geemap.add_text_to_gif(
    out_gif,
    texted_gif,
    xy=("3%", "3%"),
    text_sequence=days,
    font_size=20,
    duration=800,
    font_color="#ffffff",
    add_progress_bar=False,
)

print("====================================================")

2023-04-01 2023-05-31 60
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_Arctic_2023-04-01_2023-05-31.gif
